In [3]:
raw_matrix = '''08 02 22 97 38 15 00 40 00 75 04 05 07 78 52 12 50 77 91 08
49 49 99 40 17 81 18 57 60 87 17 40 98 43 69 48 04 56 62 00
81 49 31 73 55 79 14 29 93 71 40 67 53 88 30 03 49 13 36 65
52 70 95 23 04 60 11 42 69 24 68 56 01 32 56 71 37 02 36 91
22 31 16 71 51 67 63 89 41 92 36 54 22 40 40 28 66 33 13 80
24 47 32 60 99 03 45 02 44 75 33 53 78 36 84 20 35 17 12 50
32 98 81 28 64 23 67 10 26 38 40 67 59 54 70 66 18 38 64 70
67 26 20 68 02 62 12 20 95 63 94 39 63 08 40 91 66 49 94 21
24 55 58 05 66 73 99 26 97 17 78 78 96 83 14 88 34 89 63 72
21 36 23 09 75 00 76 44 20 45 35 14 00 61 33 97 34 31 33 95
78 17 53 28 22 75 31 67 15 94 03 80 04 62 16 14 09 53 56 92
16 39 05 42 96 35 31 47 55 58 88 24 00 17 54 24 36 29 85 57
86 56 00 48 35 71 89 07 05 44 44 37 44 60 21 58 51 54 17 58
19 80 81 68 05 94 47 69 28 73 92 13 86 52 17 77 04 89 55 40
04 52 08 83 97 35 99 16 07 97 57 32 16 26 26 79 33 27 98 66
88 36 68 87 57 62 20 72 03 46 33 67 46 55 12 32 63 93 53 69
04 42 16 73 38 25 39 11 24 94 72 18 08 46 29 32 40 62 76 36
20 69 36 41 72 30 23 88 34 62 99 69 82 67 59 85 74 04 36 16
20 73 35 29 78 31 90 01 74 31 49 71 48 86 81 16 23 57 05 54
01 70 54 71 83 51 54 69 16 92 33 48 61 43 52 01 89 19 67 48'''

In [4]:
import numpy as np

In [5]:
# %%timeit
M = np.array([int(x) for x in raw_matrix.split()]).reshape(20,20)
RM = M[::-1, ]  # flipping the array to get anti-diagonals

maxprod = 0
for i in range(20):
    for j in range(20):
        rprod = np.prod(M[i][j:j+4])
        cprod = np.prod(M[:, j][i:i+4])
        k = min(i,j)
        d1 = np.prod(M.diagonal(j-i)[k:k+4])
        d2 = np.prod(RM.diagonal(j-i)[k:k+4])
        maxprod = max(maxprod, rprod, cprod, d1, d2)
print(maxprod)
# ----------------
# 70600674
# 100 loops, best of 3: 18.7 ms per loop

70600674


# Optimized using Numba
## Caustion: Numba jit doesn't work here - it slows down the program

In [6]:
M = np.array([int(x) for x in raw_matrix.split()]).reshape(20,20)
RM = M[::-1, ]  # flipping the array to get anti-diagonals

In [7]:
from numba import jit

In [8]:
@jit
def maxprod_jit():
    maxprod = 0
    for i in range(20):
        for j in range(20):
            rprod = np.prod(M[i][j:j+4])
            cprod = np.prod(M[:, j][i:i+4])
            k = min(i,j)
            d1 = np.prod(M.diagonal(j-i)[k:k+4])
            d2 = np.prod(RM.diagonal(j-i)[k:k+4])
            maxprod = max(maxprod, rprod, cprod, d1, d2)
    return maxprod
# ----------------
# 70600674

In [9]:
%timeit maxprod_jit()
# 70600674

The slowest run took 75.39 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 16.1 ms per loop


# Optimized using Cython

In [10]:
%load_ext cython

In [11]:
%%cython
import numpy

cdef raw_matrix = '''08 02 22 97 38 15 00 40 00 75 04 05 07 78 52 12 50 77 91 08
49 49 99 40 17 81 18 57 60 87 17 40 98 43 69 48 04 56 62 00
81 49 31 73 55 79 14 29 93 71 40 67 53 88 30 03 49 13 36 65
52 70 95 23 04 60 11 42 69 24 68 56 01 32 56 71 37 02 36 91
22 31 16 71 51 67 63 89 41 92 36 54 22 40 40 28 66 33 13 80
24 47 32 60 99 03 45 02 44 75 33 53 78 36 84 20 35 17 12 50
32 98 81 28 64 23 67 10 26 38 40 67 59 54 70 66 18 38 64 70
67 26 20 68 02 62 12 20 95 63 94 39 63 08 40 91 66 49 94 21
24 55 58 05 66 73 99 26 97 17 78 78 96 83 14 88 34 89 63 72
21 36 23 09 75 00 76 44 20 45 35 14 00 61 33 97 34 31 33 95
78 17 53 28 22 75 31 67 15 94 03 80 04 62 16 14 09 53 56 92
16 39 05 42 96 35 31 47 55 58 88 24 00 17 54 24 36 29 85 57
86 56 00 48 35 71 89 07 05 44 44 37 44 60 21 58 51 54 17 58
19 80 81 68 05 94 47 69 28 73 92 13 86 52 17 77 04 89 55 40
04 52 08 83 97 35 99 16 07 97 57 32 16 26 26 79 33 27 98 66
88 36 68 87 57 62 20 72 03 46 33 67 46 55 12 32 63 93 53 69
04 42 16 73 38 25 39 11 24 94 72 18 08 46 29 32 40 62 76 36
20 69 36 41 72 30 23 88 34 62 99 69 82 67 59 85 74 04 36 16
20 73 35 29 78 31 90 01 74 31 49 71 48 86 81 16 23 57 05 54
01 70 54 71 83 51 54 69 16 92 33 48 61 43 52 01 89 19 67 48'''

cdef M = numpy.array([int(x) for x in raw_matrix.split()]).reshape(20,20)
cdef RM = M[::-1, ]  # flipping the array to get anti-diagonals

cpdef maxprod_cython():
    cdef int maxprod = 0
    for i in range(20):
        for j in range(20):
            rprod = numpy.prod(M[i][j:j+4])
            cprod = numpy.prod(M[:, j][i:i+4])
            k = min(i,j)
            d1 = numpy.prod(M.diagonal(j-i)[k:k+4])
            d2 = numpy.prod(RM.diagonal(j-i)[k:k+4])
            maxprod = max(maxprod, rprod, cprod, d1, d2)
    return maxprod

In [12]:
%timeit maxprod_cython()
# 70600674

100 loops, best of 3: 13.8 ms per loop


# Program from Project Naukai

In [15]:
# We visit each grid cell and compute the product in the 4 directions starting from that cell.
def compute():
	ans = -1
	width = len(GRID[0])
	height = len(GRID)
	for y in range(height):
		for x in range(width):
			if x + CONSECUTIVE <= width:
				ans = max(grid_product(x, y,  1, 0, CONSECUTIVE), ans)
			if y + CONSECUTIVE <= height:
				ans = max(grid_product(x, y,  0, 1, CONSECUTIVE), ans)
			if x + CONSECUTIVE <= width and y + CONSECUTIVE <= height:
				ans = max(grid_product(x, y,  1, 1, CONSECUTIVE), ans)
			if x - CONSECUTIVE >= -1    and y + CONSECUTIVE <= height:
				ans = max(grid_product(x, y, -1, 1, CONSECUTIVE), ans)
	return str(ans)


def grid_product(ox, oy, dx, dy, n):
	result = 1
	for i in range(n):
		result *= GRID[oy + i * dy][ox + i * dx]
	return result


GRID = [
	[ 8, 2,22,97,38,15, 0,40, 0,75, 4, 5, 7,78,52,12,50,77,91, 8],
	[49,49,99,40,17,81,18,57,60,87,17,40,98,43,69,48, 4,56,62, 0],
	[81,49,31,73,55,79,14,29,93,71,40,67,53,88,30, 3,49,13,36,65],
	[52,70,95,23, 4,60,11,42,69,24,68,56, 1,32,56,71,37, 2,36,91],
	[22,31,16,71,51,67,63,89,41,92,36,54,22,40,40,28,66,33,13,80],
	[24,47,32,60,99, 3,45, 2,44,75,33,53,78,36,84,20,35,17,12,50],
	[32,98,81,28,64,23,67,10,26,38,40,67,59,54,70,66,18,38,64,70],
	[67,26,20,68, 2,62,12,20,95,63,94,39,63, 8,40,91,66,49,94,21],
	[24,55,58, 5,66,73,99,26,97,17,78,78,96,83,14,88,34,89,63,72],
	[21,36,23, 9,75, 0,76,44,20,45,35,14, 0,61,33,97,34,31,33,95],
	[78,17,53,28,22,75,31,67,15,94, 3,80, 4,62,16,14, 9,53,56,92],
	[16,39, 5,42,96,35,31,47,55,58,88,24, 0,17,54,24,36,29,85,57],
	[86,56, 0,48,35,71,89, 7, 5,44,44,37,44,60,21,58,51,54,17,58],
	[19,80,81,68, 5,94,47,69,28,73,92,13,86,52,17,77, 4,89,55,40],
	[ 4,52, 8,83,97,35,99,16, 7,97,57,32,16,26,26,79,33,27,98,66],
	[88,36,68,87,57,62,20,72, 3,46,33,67,46,55,12,32,63,93,53,69],
	[ 4,42,16,73,38,25,39,11,24,94,72,18, 8,46,29,32,40,62,76,36],
	[20,69,36,41,72,30,23,88,34,62,99,69,82,67,59,85,74, 4,36,16],
	[20,73,35,29,78,31,90, 1,74,31,49,71,48,86,81,16,23,57, 5,54],
	[ 1,70,54,71,83,51,54,69,16,92,33,48,61,43,52, 1,89,19,67,48],
]
CONSECUTIVE = 4


if __name__ == "__main__":
	print(compute())

70600674


In [16]:
%timeit compute()

100 loops, best of 3: 4.1 ms per loop


# Nayukai Optimized with Numba

In [17]:
from numba import jit

In [18]:
# We visit each grid cell and compute the product in the 4 directions starting from that cell.
@jit
def compute():
	ans = -1
	width = len(GRID[0])
	height = len(GRID)
	for y in range(height):
		for x in range(width):
			if x + CONSECUTIVE <= width:
				ans = max(grid_product(x, y,  1, 0, CONSECUTIVE), ans)
			if y + CONSECUTIVE <= height:
				ans = max(grid_product(x, y,  0, 1, CONSECUTIVE), ans)
			if x + CONSECUTIVE <= width and y + CONSECUTIVE <= height:
				ans = max(grid_product(x, y,  1, 1, CONSECUTIVE), ans)
			if x - CONSECUTIVE >= -1    and y + CONSECUTIVE <= height:
				ans = max(grid_product(x, y, -1, 1, CONSECUTIVE), ans)
	return str(ans)

@jit
def grid_product(ox, oy, dx, dy, n):
	result = 1
	for i in range(n):
		result *= GRID[oy + i * dy][ox + i * dx]
	return result


GRID = [
	[ 8, 2,22,97,38,15, 0,40, 0,75, 4, 5, 7,78,52,12,50,77,91, 8],
	[49,49,99,40,17,81,18,57,60,87,17,40,98,43,69,48, 4,56,62, 0],
	[81,49,31,73,55,79,14,29,93,71,40,67,53,88,30, 3,49,13,36,65],
	[52,70,95,23, 4,60,11,42,69,24,68,56, 1,32,56,71,37, 2,36,91],
	[22,31,16,71,51,67,63,89,41,92,36,54,22,40,40,28,66,33,13,80],
	[24,47,32,60,99, 3,45, 2,44,75,33,53,78,36,84,20,35,17,12,50],
	[32,98,81,28,64,23,67,10,26,38,40,67,59,54,70,66,18,38,64,70],
	[67,26,20,68, 2,62,12,20,95,63,94,39,63, 8,40,91,66,49,94,21],
	[24,55,58, 5,66,73,99,26,97,17,78,78,96,83,14,88,34,89,63,72],
	[21,36,23, 9,75, 0,76,44,20,45,35,14, 0,61,33,97,34,31,33,95],
	[78,17,53,28,22,75,31,67,15,94, 3,80, 4,62,16,14, 9,53,56,92],
	[16,39, 5,42,96,35,31,47,55,58,88,24, 0,17,54,24,36,29,85,57],
	[86,56, 0,48,35,71,89, 7, 5,44,44,37,44,60,21,58,51,54,17,58],
	[19,80,81,68, 5,94,47,69,28,73,92,13,86,52,17,77, 4,89,55,40],
	[ 4,52, 8,83,97,35,99,16, 7,97,57,32,16,26,26,79,33,27,98,66],
	[88,36,68,87,57,62,20,72, 3,46,33,67,46,55,12,32,63,93,53,69],
	[ 4,42,16,73,38,25,39,11,24,94,72,18, 8,46,29,32,40,62,76,36],
	[20,69,36,41,72,30,23,88,34,62,99,69,82,67,59,85,74, 4,36,16],
	[20,73,35,29,78,31,90, 1,74,31,49,71,48,86,81,16,23,57, 5,54],
	[ 1,70,54,71,83,51,54,69,16,92,33,48,61,43,52, 1,89,19,67,48],
]
CONSECUTIVE = 4


if __name__ == "__main__":
	print(compute())

70600674


In [19]:
compute()

'70600674'

In [20]:
%timeit compute()

100 loops, best of 3: 16.4 ms per loop


# Optimized with Cython

In [21]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [22]:
%%cython


# We visit each grid cell and compute the product in the 4 directions starting from that cell.
cpdef compute():
	ans = -1
	width = len(GRID[0])
	height = len(GRID)
	for y in range(height):
		for x in range(width):
			if x + CONSECUTIVE <= width:
				ans = max(grid_product(x, y,  1, 0, CONSECUTIVE), ans)
			if y + CONSECUTIVE <= height:
				ans = max(grid_product(x, y,  0, 1, CONSECUTIVE), ans)
			if x + CONSECUTIVE <= width and y + CONSECUTIVE <= height:
				ans = max(grid_product(x, y,  1, 1, CONSECUTIVE), ans)
			if x - CONSECUTIVE >= -1    and y + CONSECUTIVE <= height:
				ans = max(grid_product(x, y, -1, 1, CONSECUTIVE), ans)
	return str(ans)


cpdef grid_product(ox, oy, dx, dy, n):
	result = 1
	for i in range(n):
		result *= GRID[oy + i * dy][ox + i * dx]
	return result


GRID = [
	[ 8, 2,22,97,38,15, 0,40, 0,75, 4, 5, 7,78,52,12,50,77,91, 8],
	[49,49,99,40,17,81,18,57,60,87,17,40,98,43,69,48, 4,56,62, 0],
	[81,49,31,73,55,79,14,29,93,71,40,67,53,88,30, 3,49,13,36,65],
	[52,70,95,23, 4,60,11,42,69,24,68,56, 1,32,56,71,37, 2,36,91],
	[22,31,16,71,51,67,63,89,41,92,36,54,22,40,40,28,66,33,13,80],
	[24,47,32,60,99, 3,45, 2,44,75,33,53,78,36,84,20,35,17,12,50],
	[32,98,81,28,64,23,67,10,26,38,40,67,59,54,70,66,18,38,64,70],
	[67,26,20,68, 2,62,12,20,95,63,94,39,63, 8,40,91,66,49,94,21],
	[24,55,58, 5,66,73,99,26,97,17,78,78,96,83,14,88,34,89,63,72],
	[21,36,23, 9,75, 0,76,44,20,45,35,14, 0,61,33,97,34,31,33,95],
	[78,17,53,28,22,75,31,67,15,94, 3,80, 4,62,16,14, 9,53,56,92],
	[16,39, 5,42,96,35,31,47,55,58,88,24, 0,17,54,24,36,29,85,57],
	[86,56, 0,48,35,71,89, 7, 5,44,44,37,44,60,21,58,51,54,17,58],
	[19,80,81,68, 5,94,47,69,28,73,92,13,86,52,17,77, 4,89,55,40],
	[ 4,52, 8,83,97,35,99,16, 7,97,57,32,16,26,26,79,33,27,98,66],
	[88,36,68,87,57,62,20,72, 3,46,33,67,46,55,12,32,63,93,53,69],
	[ 4,42,16,73,38,25,39,11,24,94,72,18, 8,46,29,32,40,62,76,36],
	[20,69,36,41,72,30,23,88,34,62,99,69,82,67,59,85,74, 4,36,16],
	[20,73,35,29,78,31,90, 1,74,31,49,71,48,86,81,16,23,57, 5,54],
	[ 1,70,54,71,83,51,54,69,16,92,33,48,61,43,52, 1,89,19,67,48],
]
CONSECUTIVE = 4


# if __name__ == "__main__":
# 	print(compute())

In [23]:
compute()

'70600674'

In [24]:
%timeit compute()

100 loops, best of 3: 2 ms per loop
